In [8]:
import pandas as pd
df=messages = pd.read_csv('SMSSpamCollection', sep='\t',
                           names=["label", "message"])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
df.shape

(5572, 2)

In [10]:
#independent features
X=list(df['message'])

In [11]:
y=list(df['label'])

In [12]:
y=list(pd.get_dummies(y,drop_first=True)['spam'])

In [13]:
### Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [14]:
!pip install tokenizers
!pip install --no-build-isolation --no-deps transformers

1.   call the pretrained model(Distilbert)
2.   call the tokenizer(Distilbert)


In [15]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [16]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [17]:
### Conver these encodings into Dataset objects.
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).batch(8)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(8)

In [19]:
from transformers import TFDistilBertForSequenceClassification, AutoTokenizer
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ["accuracy"]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Train the model
model.fit(train_dataset, epochs=3)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
558/558 [==============================] - 144s 199ms/step - loss: 0.0693 - accuracy: 0.9803
Epoch 2/3
558/558 [==============================] - 112s 200ms/step - loss: 0.0231 - accuracy: 0.9944
Epoch 3/3
558/558 [==============================] - 112s 200ms/step - loss: 0.0094 - accuracy: 0.9973


In [20]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


140/140 [==============================] - 14s 72ms/step - loss: 0.0446 - accuracy: 0.9937
Test Loss: 0.0446
Test Accuracy: 0.9937


In [21]:
import numpy as np

# Get model predictions
predictions = model.predict(test_dataset)

# Convert logits to class labels
predicted_labels = np.argmax(predictions.logits, axis=1)

print(predicted_labels[:10])  # Show first 10 predictions


140/140 [==============================] - 12s 66ms/step
[0 1 0 0 0 0 1 0 0 0]


In [22]:
from sklearn.metrics import classification_report

# True labels
true_labels = np.concatenate([y for x, y in test_dataset], axis=0)

# Print classification report
print(classification_report(true_labels, predicted_labels))


              precision    recall  f1-score   support

       False       0.99      1.00      1.00       955
        True       0.99      0.97      0.98       160

    accuracy                           0.99      1115
   macro avg       0.99      0.98      0.99      1115
weighted avg       0.99      0.99      0.99      1115



In [23]:
model.save("distilbert_model")

In [24]:
from tensorflow.keras.models import load_model

loaded_model = load_model("distilbert_model")


ValueError: File format not supported: filepath=distilbert_model. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(distilbert_model, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).